In [2]:
import numpy as np

In [3]:
import cv2
import numpy as np
from mtcnn import MTCNN
import joblib
import torch
from facenet_pytorch import InceptionResnetV1
import torchvision.transforms as transforms
import os
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
knn_model = joblib.load(r"D:\GitHub\Machine_Learning\weights\knn_model_ex_optimal.joblib")

# Danh sách nhãn
labels = ['Thang', 'Su', 'Nhung', 'Tuyen', 'Vu', 'Dat', 'Huy']

# Khởi tạo MTCNN để phát hiện khuôn mặt
detector = MTCNN()

# Khởi tạo mô hình InceptionResnetV1 để trích xuất đặc trưng khuôn mặt
feature_extractor = InceptionResnetV1(pretrained='vggface2', classify=False).eval()

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
image_path = r"C:\Users\Nguyen Van Thang\Pictures\Camera Roll\WIN_20250623_17_00_18_Pro.jpg"  # Thay đổi đường dẫn ảnh của bạn
image = cv2.imread(image_path)

# Chuyển ảnh từ BGR sang RGB (MTCNN yêu cầu ảnh RGB)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Phát hiện khuôn mặt bằng MTCNN
faces = detector.detect_faces(image_rgb)

# Ngưỡng tương tự để phân biệt khuôn mặt unknown (ví dụ: 0.6)
similarity_threshold = 0.8

In [7]:
import cv2
import numpy as np
import torch
import os
from sklearn.metrics.pairwise import cosine_similarity
from mtcnn import MTCNN

# Giả sử bạn đã có:
# - feature_extractor
# - knn_model
# - labels
# - transform
# - similarity_threshold

detector = MTCNN()
cap = cv2.VideoCapture(0)

last_frame = None  # Lưu frame cuối

while True:
    ret, frame = cap.read()
    if not ret:
        break

    image = frame.copy()
    faces = detector.detect_faces(image)

    if faces:
        for face in faces:
            x, y, w, h = face['box']
            face_crop = image[y:y+h, x:x+w]
            if face_crop.shape[0] == 0 or face_crop.shape[1] == 0:
                continue

            face_resized = cv2.resize(face_crop, (160, 160))
            face_tensor = transform(face_resized).unsqueeze(0)

            with torch.no_grad():
                face_features = feature_extractor(face_tensor)

            face_features = face_features.cpu().numpy().flatten().reshape(1, -1)
            knn_label_idx = knn_model.predict(face_features)[0]
            knn_label = labels[knn_label_idx]

            distances = cosine_similarity(face_features, knn_model._fit_X)
            if np.max(distances) < similarity_threshold:
                knn_label = "Unknown"

            print(f'Predicted label: {knn_label}')
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, f'KNN: {knn_label}', (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    else:
        cv2.putText(image, 'No face detected', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    last_frame = image.copy()
    cv2.imshow('Face Recognition', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Lưu kết quả cuối cùng
if last_frame is not None:
    output_dir = r'D:\GitHub\Machine_Learning\output\knn_ex'
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, 'output_image_knn.jpg')
    cv2.imwrite(output_path, last_frame)
    print(f'Result saved to {output_path}')
    cv2.imshow("Detected Face", last_frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("No frame was captured to save.")


Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Unknown
Predicted label: Thang
Predicted label: Unknown
Predicted label: Unknown
Predicted label: Thang
Predicted label: Thang
Predicted label: Thang
Predicted label: Tuyen
Predicted label: Thang
Predicted label: Tuyen
Predicted label: Tuyen
Predicted label: Thang
Predicted label: Tuyen
Predicted label: Thang
Predicted label: Tuyen
Predicted label: Thang
Predicted label: Tuyen
Predicted label: Thang
Predicted label: Tuyen
Predicted label: Thang
Predicted label: Thang
Predicted label: Unknown
Predicted label: Thang
Predicted label: Tuyen
Predicted label: Thang
Predicted label: Tuyen
Predicted label: Unknown
Predicted label: Thang
P